<a href="https://colab.research.google.com/github/James606240/NTUT_Test/blob/main/GT_CH9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import tensorflow as tf

def parse_aug_fn(dataset):
    """
    Image Augmentation(影像增強) function
    """
    x = tf.cast(dataset['image'], tf.float32) / 255.  # 影像標準化
    x = flip(x)  # 隨機水平翻轉
    # 觸發顏色轉換機率50%
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: color(x), lambda: x)
    # 觸發影像旋轉機率0.25%
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.75, lambda: rotate(x), lambda: x)
    # 觸發影像縮放機率50%
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: zoom(x), lambda: x)
    # 將輸出標籤轉乘One-hot編碼
    y = tf.one_hot(dataset['label'], 10)
    return x, y


def parse_fn(dataset):
    x = tf.cast(dataset['image'], tf.float32) / 255.  # 影像標準化
    # 將輸出標籤轉乘One-hot編碼
    y = tf.one_hot(dataset['label'], 10)
    return x, y


def flip(x):
    """
    flip image(翻轉影像)
    """
    x = tf.image.random_flip_left_right(x)  # 隨機左右翻轉影像
    return x


def color(x):
    """
     Color change(改變顏色)
    """
    x = tf.image.random_hue(x, 0.08)  # 隨機調整影像色調
    x = tf.image.random_saturation(x, 0.6, 1.6)  # 隨機調整影像飽和度
    x = tf.image.random_brightness(x, 0.05)  # 隨機調整影像亮度
    x = tf.image.random_contrast(x, 0.7, 1.3)  # 隨機調整影像對比度
    return x


def rotate(x):
    """
    Rotation image(影像旋轉)
    """
    # 隨機選轉n次(通過minval和maxval設定n的範圍)，每次選轉90度
    x = tf.image.rot90(x,tf.random.uniform(shape=[],minval=1,maxval=4,dtype=tf.int32))
    return x


def zoom(x, scale_min=0.6, scale_max=1.4):
    """
    Zoom Image(影像縮放)
    """
    h, w, c = x.shape
    scale = tf.random.uniform([], scale_min, scale_max)  # 隨機縮放比例
    sh = h * scale  # 縮放後影像長度
    sw = w * scale  # 縮放後影像寬度
    x = tf.image.resize(x, (sh, sw))  # 影像縮放
    x = tf.image.resize_with_crop_or_pad(x, h, w)  # 影像裁減和填補
    return x

In [13]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
# 從資料夾中的preprocessing.py檔案中Import flip, color, rotate, zoom影像增強函數
# from preprocessing import flip, color, rotate, zoom

In [14]:
# 將train Data重新分成8:1:1等分，分別分給train data, valid data, test data
train_split, valid_split, test_split = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']
# 取得訓練數據，並順便讀取data的資訊
train_data, info = tfds.load("cats_vs_dogs", split=train_split, with_info=True)
# 取得驗證數據
valid_data = tfds.load("cats_vs_dogs", split=valid_split)
# 取得測試數據
test_data = tfds.load("cats_vs_dogs", split=test_split)

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteMGAHHQ/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [15]:
print(info.features['label'].names)
decoder = info.features['label'].names

['cat', 'dog']


In [16]:
input_shape = (224, 224)

In [17]:
def parse_aug_fn(dataset):
    """
    Image Augmentation(影像增強) function
    """
    x = tf.cast(dataset['image'], tf.float32) / 255.  # 影像標準化
    x = tf.image.resize(x, input_shape)
    x = flip(x)  # 隨機水平翻轉
    # 觸發顏色轉換機率50%
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: color(x), lambda: x)
    # 觸發影像旋轉機率0.25%
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.75, lambda: rotate(x), lambda: x)
    # 觸發影像縮放機率50%
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: zoom(x), lambda: x)
    return x, dataset['label']

def parse_fn(dataset):
    x = tf.cast(dataset['image'], tf.float32) / 255.  # 影像標準化
    x = tf.image.resize(x, input_shape)
    return x, dataset['label']

In [18]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
buffer_size = 1000  # 因為這次的影像較大，緩存空間設1000就好
bacth_size = 32  # 批次大小

# 載入預處理「 parse_aug_fn」function，cpu數量為自動調整模式
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.shuffle(buffer_size)  # 打散資料集
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
train_data = train_data.batch(bacth_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
valid_data = valid_data.batch(bacth_size).prefetch(buffer_size=AUTOTUNE)

# 載入預處理「 parse_fn」function，cpu數量為自動調整模式
test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# 設定批次大小並將prefetch模式開啟(暫存空間為自動調整模式)
test_data = test_data.batch(bacth_size).prefetch(buffer_size=AUTOTUNE)

In [19]:
# !rm -r lab8-logs  # 移除目錄(以防萬一)
model_dir = 'lab9-logs/models'  # 設定儲存權重目錄
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)  # 創建儲存權重目錄

In [20]:
# 儲存訓練記錄檔
log_dir = os.path.join('lab9-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
# 儲存最好的網路模型權重
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/Best-model-1.h5', 
                                             monitor='val_binary_accuracy', 
                                             save_best_only=True, 
                                             mode='max')
# 設定停止訓練的條件(當Accuracy超過30迭代沒有上升的話訓練會終止)
model_esp = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=20)

In [21]:
# 創建模型，最後一層卷積加上GlobalAveragePooling
base_model = tf.keras.applications.EfficientNetB0(include_top=False, 
                                               pooling='avg', 
                                               input_shape=input_shape+(3,))
# 將剛創建的EfficientNet B0模型接上兩層全連接層，並且最後一層使用Sigmoid輸出
model_1 = tf.keras.Sequential([
    base_model,
    layers.Dense(128, activation='relu'), 
    layers.Dense(1, activation='sigmoid')
])

16711680/16705208 [==============================] - 0s 0us/step


In [22]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 4,213,668
Trainable params: 4,171,645
Non-trainable params: 42,023
_________________________________________________________________


In [23]:
model_1.compile(keras.optimizers.Adam(), 
                loss=keras.losses.BinaryCrossentropy(), 
                metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
history = model_1.fit(train_data,
                      epochs=1, 
                      validation_data=valid_data,
                      callbacks=[model_cbk, model_mckp, model_esp])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


  3/582 [..............................] - ETA: 1:19:43 - loss: 0.6348 - binary_accuracy: 0.6458

In [ ]:
# 儲存訓練記錄檔
log_dir = os.path.join('lab9-logs', 'model-2')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
# 儲存最好的網路模型權重
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/Best-model-2.h5', 
                                             monitor='val_binary_accuracy', 
                                             save_best_only=True, 
                                             mode='max')
# 設定停止訓練的條件(當Accuracy超過30迭代沒有上升的話訓練會終止)
model_esp = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=20, mode='max')

In [ ]:
# 權重檔網址
module_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

model_2 = tf.keras.Sequential([
    # hub.KerasLayer載入的模型為Keras Layer
    hub.KerasLayer(module_url,
                   input_shape=(224, 224, 3),  # 模型輸入大小
                   trainable=False),  # 將模型訓練權重設定為False(凍結)
    # 最後接上兩層全連接層，並且輸出使用Sigmoid
    layers.Dense(128, activation='relu'), 
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(keras.optimizers.Adam(), 
               loss=keras.losses.BinaryCrossentropy(), 
               metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
history = model_2.fit(train_data,
                      epochs=1, 
                      validation_data=valid_data,
                      callbacks=[model_cbk, model_esp, model_mckp])

In [ ]:
model_1.load_weights(model_dir + '/Best-model-1.h5')
model_2.load_weights(model_dir + '/Best-model-2.h5')

In [ ]:
:loss_1, acc_1 = model_1.evaluate(test_data)
loss_2, acc_2 = model_2.evaluate(test_data)

In [ ]:
print("Model_1 Prediction: {}%".format(acc_1 * 100))
print("Model_2 Prediction: {}%".format(acc_2 * 100))

In [ ]:
acc_2 - acc_1